In [38]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()
import chardet  # ! pip install chardet

In [39]:
def bundesland(k):
    if k == 'Sa':
        return 'Salzburg'
    elif k == 'St':
        return 'Steiermark'
    elif k == 'W':
        return 'Wien'
    elif k == 'T':
        return 'Tirol'
    elif k == 'V':
        return 'Vorarlberg'
    elif k == 'K':
        return 'Kärnten'
    elif k == 'O':
        return 'Oberösterreich'
    elif k == 'N':
        return 'Niederösterreich'
    elif k == 'B':
        return 'Burgenland'

In [40]:
with open(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
tmp = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), sep=';', encoding=result['encoding'])
tmp.rename(columns={'GEMNR': 'GKZ'}, inplace=True)
print(f"PLZ_BESTIMMUNGSORT-20220629.csv len: {tmp.shape[0]}, unique PLZ: {tmp.PLZ.nunique()}, duplicates: {tmp.duplicated('PLZ', keep=False).sum()}")
#tmp.head(5)

PLZ_BESTIMMUNGSORT-20220629.csv len: 18867, unique PLZ: 2228, duplicates: 18432


In [41]:
# tmp[tmp.duplicated('PLZ', keep=False) == True]
df = tmp.groupby(['PLZ'], as_index=False).agg({'PLZ':'first', 'Bestimmungsort':'first', 'OKZ':'first', 'Ortschaft':'first', 'GKZ':'first', 'GEMNAM': 'first'})
print(f"len: {df2.shape[0]}, unique PLZ: {df2.PLZ.nunique()}, PLZ duplicates: {df2.duplicated('PLZ', keep=False).sum()}")
df.head(3)

len: 2228, unique PLZ: 2228, PLZ duplicates: 0


,PLZ,Bestimmungsort,OKZ,Ortschaft,GKZ,GEMNAM
0,1010,Wien,17223,"Wien,Innere Stadt",90001,Wien
1,1020,Wien,17224,"Wien,Leopoldstadt",90001,Wien
2,1030,Wien,17225,"Wien,Landstraße",90001,Wien


## Problem (Why all the hassle)?

PLZ_BESTIMMUNGSORT is missing the Ort and the Bundesland <br>
PLZ_Verzeichnis is missing the **GKZ** which is needed for mapping to the **geojson** file.

In [42]:
with open(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
plz = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_Verzeichnis-20220629.csv'), sep=';', encoding=result['encoding'])
print(f"PLZ_Verzeichnis-20220629.csv len: {plz.shape[0]}, unique PLZ: {plz.PLZ.nunique()}, PLZ duplicates: {plz.duplicated('PLZ', keep=False).sum()}")
plz.head(3)

PLZ_Verzeichnis-20220629.csv len: 2521, unique PLZ: 2521, PLZ duplicates: 0


,PLZ,Ort,Bundesland,gültig ab,gültig bis,NamePLZTyp,intern_extern,adressierbar,Postfach
0,1000,Wien,W,01.08.2009,NaN,PLZ-Postfach,extern,Nein,Ja
1,1004,Wien,W,01.01.1966,NaN,InteressentenPLZ,extern,Nein,Ja
2,1006,Wien,W,01.02.2015,NaN,PLZ-Postfach,extern,Nein,Ja


In [43]:
df = df.merge(plz[['PLZ','Ort', 'Bundesland']], on=['PLZ'], how='left')
print(f"Merge len: {df.shape[0]}, unique PLZ: {df.PLZ.nunique()}, duplicates: {df.duplicated('PLZ', keep=False).sum()}")
df['Bundesland'] = df.apply(lambda x: bundesland(x.Bundesland), axis=1)
df.to_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_LOOKUP_MERGE.csv'), encoding='utf-8', index=False)
df.tail(20)

Merge len: 2228, unique PLZ: 2228, duplicates: 0


,PLZ,Bestimmungsort,OKZ,Ortschaft,GKZ,GEMNAM,Ort,Bundesland
2208,9919,Heinfels,16856,Panzendorf,70735,Heinfels,Heinfels,Tirol
2209,9920,Sillian,16875,Arnbach,70728,Sillian,Sillian,Tirol
2210,9931,Außervillgraten,16780,Außervillgraten,70706,Außervillgraten,Außervillgraten,Tirol
2211,9932,Innervillgraten,16795,Innervillgraten,70710,Innervillgraten,Innervillgraten,Tirol
2212,9941,Kartitsch,16809,Hollbruck,70713,Kartitsch,Kartitsch,Tirol
2213,9942,Obertilliach,16852,Bergen,70721,Obertilliach,Obertilliach,Tirol
2214,9943,Untertilliach,16882,Untertilliach,70733,Untertilliach,Untertilliach,Tirol
2215,9951,Ainet,16763,Ainet,70702,Ainet,Ainet,Tirol
2216,9952,St. Johann im Walde,16867,St. Johann im Walde,70725,St. Johann im Walde,St. Johann im Walde,Tirol
2217,9954,Schlaiten,16874,Schlaiten,70727,Schlaiten,Schlaiten,Tirol
